# PLANEJAMENTO E ENTEDIMENTO DO NEGÓCIO

**A empresa International Bank**

    Disclaimer: O contexto a seguir é completamente fictício, a empresa, o contexto, o CEO e as perguntas de negócio existem somente na minha imaginação.

    O International Bank é um banco internacional que fornece produtos financeiros, como empréstimos, contas correntes, contas poupança, opções de investimentos, cartões de crédito e etc.

    A cada quarter do ano, os líderes se reúnem para decidir os objetivos da empresa para os próximos 3 meses. Nessa última reunião, uma das metas decididas é a criação de uma estratégia de marketing para abordar os clientes que usam cartão de crédito.

    A empresa possui um conjunto de dados sobre o comportamento de uso de aproximadamente 9000 usuários de cartão de crédito ativos, durante os últimos 6 meses e o arquivo está no nível de clientes e possui 18 variáveis comportamentais.

**O desafio**

    Você foi contratado como um cientista de dados consultor, para criar um produto de dados que cria uma segmentação de clientes que direcionará a estratégia de marketing e os resultados esperados do seu trabalho de consultoria, são os seguintes:

    - Criar uma segmentação de clientes baseado em critérios claros de seleção.
    - Tornar a segmentação de clientes acessível para o time de operações.
    - Ensinar o time de operações sobre como usar a sua solução.
    - Transferir o conhecimento para os Cientistas de Dados da International Bank.
    - Recomendar 2-3 alavancas de negócio para cada segmento de clientes.

**Roteiro**

    1 - Explore os Dados usando ferramentas e testes Estatísticos com o objetivo de encontrar inconsistências dos dados e tratar possíveis dados faltantes.
    2 - Levante Hipóteses sobre as características de um cliente Insider.
    3 - Prepare os dados para ensinar os algoritmos de Machine Learning.
    4 - Escreva vários algoritmos de Agrupamento.
    5 - Contabilize a performance do modelo de Machine Learning treinado e valide a segmentação definida.
    6 - Monte um sistema de fornecimento da segmentação na Cloud da AWS e use uma ferramenta de visualização para acessar os dados.
    7 - Crie um README sobre como interpretar e usar a sua solução.
    8 - Escreva um Relatório com as recomendações das alavancas para cada segmento.

**RFM Model**
    
    1. Recency
        a. Tempo desde a última compra.
        b. Responsividade
        
    2. Frequency
        a. Tempo médio entre as transações
        b. Engajamento
        
    3. Monetary
        a. Total gasto, faturamento
        b. "High-value Purchases"

## IOT

### Input

### Output

### Tasks

# IMPORTS

In [55]:
import inflection

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from IPython.core.display  import HTML

from sklearn import cluster as c
from sklearn import metrics as m

pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Helper Functions

In [56]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()
    
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


# LOADING DATA 

In [57]:
data = pd.read_csv('datasets/bankmarketing.csv')

# DATA DESCRIPTION

## Rename Columns

In [58]:
data.columns = data.columns.str.lower()

## Head

In [59]:
data.head()

,cust_id,balance,balance_frequency,purchases,oneoff_purchases,installments_purchases,cash_advance,purchases_frequency,oneoff_purchases_frequency,purchases_installments_frequency,cash_advance_frequency,cash_advance_trx,purchases_trx,credit_limit,payments,minimum_payments,prc_full_payment,tenure
0,C10001,40.90,0.82,95.40,0.00,95.40,0.00,0.17,0.00,0.08,0.00,0,2,1000.00,201.80,139.51,0.00,12
1,C10002,3202.47,0.91,0.00,0.00,0.00,6442.95,0.00,0.00,0.00,0.25,4,0,7000.00,4103.03,1072.34,0.22,12
2,C10003,2495.15,1.00,773.17,773.17,0.00,0.00,1.00,1.00,0.00,0.00,0,12,7500.00,622.07,627.28,0.00,12
3,C10004,1666.67,0.64,1499.00,1499.00,0.00,205.79,0.08,0.08,0.00,0.08,1,1,7500.00,0.00,NaN,0.00,12
4,C10005,817.71,1.00,16.00,16.00,0.00,0.00,0.08,0.08,0.00,0.00,0,1,1200.00,678.33,244.79,0.00,12


## Columns

In [60]:
data.columns

Index(['cust_id', 'balance', 'balance_frequency', 'purchases',
       'oneoff_purchases', 'installments_purchases', 'cash_advance',
       'purchases_frequency', 'oneoff_purchases_frequency',
       'purchases_installments_frequency', 'cash_advance_frequency',
       'cash_advance_trx', 'purchases_trx', 'credit_limit', 'payments',
       'minimum_payments', 'prc_full_payment', 'tenure'],
      dtype='object')

## Data Dimension

In [61]:
print(f'Essa base tem {data.shape[0]} linhas e {data.shape[1]} colunas')

Essa base tem 8950 linhas e 18 colunas


## Dtypes

In [62]:
data.dtypes

cust_id                              object
balance                             float64
balance_frequency                   float64
purchases                           float64
oneoff_purchases                    float64
installments_purchases              float64
cash_advance                        float64
purchases_frequency                 float64
oneoff_purchases_frequency          float64
purchases_installments_frequency    float64
cash_advance_frequency              float64
cash_advance_trx                      int64
purchases_trx                         int64
credit_limit                        float64
payments                            float64
minimum_payments                    float64
prc_full_payment                    float64
tenure                                int64
dtype: object

## Check NA

In [63]:
data.isna().sum()

cust_id                               0
balance                               0
balance_frequency                     0
purchases                             0
oneoff_purchases                      0
installments_purchases                0
cash_advance                          0
purchases_frequency                   0
oneoff_purchases_frequency            0
purchases_installments_frequency      0
cash_advance_frequency                0
cash_advance_trx                      0
purchases_trx                         0
credit_limit                          1
payments                              0
minimum_payments                    313
prc_full_payment                      0
tenure                                0
dtype: int64

## Drop NA

In [64]:
data = data.dropna( subset=['minimum_payments', 'credit_limit'])

## Estatística Descritiva

In [65]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
num_attributes = data.select_dtypes(include=['int64', 'float64'])

#Central Tendecy - média e mediana
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

#Dispersion - desvio, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( min ) ).T
d3 = pd.DataFrame( num_attributes.apply( max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

#concat
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes', 'min','max','range','mean','median','std','skew','kurtosis']
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,balance,0.00,19043.14,19043.14,1601.22,916.86,2095.45,2.37,7.55
1,balance_frequency,0.00,1.00,1.00,0.90,1.00,0.21,-2.08,3.37
2,purchases,0.00,49039.57,49039.57,1025.43,375.40,2166.98,8.06,108.68
3,oneoff_purchases,0.00,40761.25,40761.25,604.90,45.00,1684.21,9.94,160.12
4,installments_purchases,0.00,22500.00,22500.00,420.84,94.78,917.19,7.22,94.19
5,cash_advance,0.00,47137.21,47137.21,994.18,0.00,2121.34,5.14,52.14
6,purchases_frequency,0.00,1.00,1.00,0.50,0.50,0.40,0.03,-1.64
7,oneoff_purchases_frequency,0.00,1.00,1.00,0.21,0.08,0.30,1.50,1.06
8,purchases_installments_frequency,0.00,1.00,1.00,0.37,0.17,0.40,0.49,-1.42
9,cash_advance_frequency,0.00,1.50,1.50,0.14,0.00,0.20,1.80,3.18


# DATA FILTERING

# FEATURE ENGINEERING 

# EDA 

# DATA PREPARATION 

# FEATURE SELECTION 

In [66]:
cols_drop = ['cust_id']
X = data.drop( columns=cols_drop )

# HYPERPARAMETER FINE TUNNING 

## K-Means

In [76]:
clusters = [2, 3, 4, 5, 6, 7, 8]

silhouette_list = []
davies_list = []
for k in clusters:
    #model definition
    kmeans_model = c.KMeans( n_clusters=k )
    
    #model training
    kmeans_model.fit( X )
    
    #model predict
    labels = kmeans_model.predict( X )
    
    #model performance
    sil = m.silhouette_score( X, labels)
    dav = m.davies_bouldin_score( X, labels)
    silhouette_list.append(sil)
    davies_list.append(dav)
    

AttributeError: 'DataFrame' object has no attribute 'davies_bouldin_score'

In [ ]:
plt.plot( clusters, silhouette_list, linestyle='--', marker='o', color='b')
plt.xlabel( 'K - Number of Clusters' )
plt.ylabel( 'SS - Silhouette Score')
plt.title( 'Silhouette Score');

In [ ]:
plt.plot( clusters, davies_list, linestyle='--', marker='o', color='b')
plt.xlabel( 'K - Number of Clusters' )
plt.ylabel( 'DB Score')
plt.title( 'DB Index');

# MACHINE LEARNING

# CLUSTERING ANALYSIS

# EDA - INSIGHT REPORTS

# DEPLOY TO PRODUCTION